In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Data/552/'

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import math
import numpy as np

In [ ]:
## Polution levels for date and grid id
training_set = pd.read_csv(path+'train_labels.csv')
training_set.head(3)

,datetime,grid_id,value
0,2018-02-01T08:00:00Z,3S31A,11.4
1,2018-02-01T08:00:00Z,A2FBI,17.0
2,2018-02-01T08:00:00Z,DJN0F,11.1


In [ ]:
training_set[training_set['grid_id'] == '3S31A']

,datetime,grid_id,value
0,2018-02-01T08:00:00Z,3S31A,11.400000
49,2018-02-03T08:00:00Z,3S31A,27.200000
79,2018-02-04T08:00:00Z,3S31A,19.844444
110,2018-02-05T08:00:00Z,3S31A,10.600000
141,2018-02-06T08:00:00Z,3S31A,20.300000
...,...,...,...
34129,2020-12-27T08:00:00Z,3S31A,5.818519
34162,2020-12-28T08:00:00Z,3S31A,3.038889
34199,2020-12-29T08:00:00Z,3S31A,8.125397
34236,2020-12-30T08:00:00Z,3S31A,10.889474


In [ ]:
#meta data for grid id
metadata = pd.read_csv(path+'grid_metadata.csv')

In [ ]:
metadata_taipei = metadata[metadata['location'] == 'Taipei']
metadata_delhi = metadata[metadata['location'] == 'Delhi']
metadata_la = metadata[metadata['location'] == 'Los Angeles (SoCAB)']

In [ ]:
metadata_la.head(3)

,grid_id,location,tz,wkt
2,3S31A,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-117.9338248256995 33.79558357488509...
11,A2FBI,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-117.3948356552278 33.98201108613195...
18,DHO4M,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-118.3380667035533 34.16803061743935...


In [ ]:
assert (len(metadata_taipei)+len(metadata_delhi)+len(metadata_la)) == len(metadata)

In [ ]:
def parse_polygon_coords(location):
    '''Parse longitude and latitude from string
    Args: 
        location: location string
    Returns: 
        list of (lat,long) polygon vertices
    '''
    coordinates = re.findall('\d*\.?\d+',location)
    coordinates = [(float(coordinates[i]), float(coordinates[i+1])) for i in range(0,len(coordinates)-1,2)]
    return coordinates

In [ ]:
def get_centroid(pc):
    '''
    https://en.wikipedia.org/wiki/Centroid#Of_a_polygon
    Args: 
        pc: list of tuples of polygon vertices
    Return: 
        (long,lat) of polygon center coordinates
    '''
    A = 0
    for i in range(len(pc)-1):
        a = pc[i][0]*pc[i+1][1] - pc[i+1][0]*pc[i][1] 
        A += a
    A *= 0.5
    Cx, Cy = 0,0
    for i in range(len(pc)-1):
        Cx += (pc[i][0]+pc[i+1][0])*(pc[i][0]*pc[i+1][1] - pc[i+1][0]*pc[i][1])
        Cy += (pc[i][1]+pc[i+1][1])*(pc[i][0]*pc[i+1][1] - pc[i+1][0]*pc[i][1])
    Cx /= 6*A
    Cy /= 6*A
    return (Cx, Cy)


In [ ]:
loc = metadata_la['wkt'].iloc[0]
parse_polygon_coords(loc)

[(117.9338248256995, 33.79558357488509),
 (117.9338248256995, 33.83290166381627),
 (117.9787405899055, 33.83290166381627),
 (117.9787405899055, 33.79558357488509),
 (117.9338248256995, 33.79558357488509)]

In [ ]:
#documentation: https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc%3AC00532/html
#sample weather data for Burbank, LA
df = pd.read_csv(path+'test.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (22,23,24,25,26,27,40,45,74) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
def clean_weather_data_columns(data, remove_cols):
    #TO-DO: Explore what the columns mean and what can be removed, 
    # and if columns with much missing data are important
    '''
    Removes irrelevant columns and columns with many missing values
    for historical weather dataset. 
    Args: 
        data: weather dataframe
        remove_cols: columns to remove
    Returns: 
        Cleaned dataframe
    '''
    data = data.drop(remove_cols, axis = 1)
    to_drop = []
    for col in data:
        if data[col].isna().sum() > 0.7 * len(col):
            data = data.drop(to_drop, axis = 1)
    return data

In [ ]:
remove_cols = ['STATION', 'SOURCE', 'NAME', 'REPORT_TYPE', 'CALL_SIGN', 'QUALITY_CONTROL']
df = clean_weather_data_columns(df, remove_cols)

In [ ]:
def group_dates(data):
    '''
    Weather data contains hourly samples for each day. Group samples
    into a dict of dataframe of hourly samples for each day. 
    Args: 
        data: weather dataframe
    Returns: 
        dict of weather dataframes with (key, val) = (date, dataframe) 
    '''
    date_indices = {}
    data = data.sort_values(by = ['DATE'])
    dates = data['DATE']
    current_date = dates.iloc[0][0:10]
    date_indices[current_date] = [0]
    #group data from same date into date-keyed dict with list
    # of corresponding indices in original df
    for i,row in enumerate(dates):
        date = row[0:10]
        if date != current_date:
            current_date = date
            date_indices[date] = []
        date_indices[date].append(i) 
    date_grouped_dfs = {}
    #for each date, slice the original df to date-keyed dict of dfs
    for key, value in date_indices.items():
        date_df = data.iloc[value]
        date_df = date_df.reset_index(drop = True)
        date_grouped_dfs[key] = date_df
    return date_grouped_dfs
        

In [ ]:
df_dict = group_dates(df)

Documentation for data: 
    Documentation of NOAA data:  
    https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc%3AC00532/html

Contains info on columns and how they should be interpreted

In [ ]:
def clean_wind_data(data):
    '''
    Parses wind data string into relevant fields. Creates new columns in weather dataframe
    Removes samples with corrupted wind data, or missing wind data. 
    'Wind direction: 0-360 degs
    'Wind speed: m/s
    '''
    wind_direction, wind_speed = [], []
    acceptable_quality = (0,1,4,5,9) #see docs
    missing_dir, missing_spd = 999, 9999
    for i,wind in enumerate(data['WND']):
        quality_dir, quality_spd = int(wind[4]), int(wind[13])
        direction = int(wind[0:3])
        speed = int(wind[8:12])
        #ID erroneuous or missing.
        if (quality_dir not in acceptable_quality or direction == missing_dir):
            direction = np.nan
        if (quality_spd not in acceptable_quality or speed == missing_spd):
            speed = np.nan
        wind_direction.append(direction), wind_speed.append(speed)
    data['Wind_Speed'] = wind_speed
    data['Wind_Direction'] = wind_direction
    return data

In [ ]:
def clean_ceiling_height_data(data):
    '''
    The height above ground level (AGL) of the lowest cloud or obscuring
    phenomena layer aloft with 5/8 or more summation total sky cover,
    which may be predominantly opaque, or the vertical visibility into a
    surface-based obstruction. Unlimited = 22000.
    height (m) above ground level of lowest cloud (unlimited = 22000)
    '''
    ceiling_height = []
    acceptable_quality = (0,1,4,5,9) #see docs
    missing = 99999
    for i,cig in enumerate(data['CIG']):
        height = int(cig[0:5])
        quality = int(cig[6])
        #ID erroneuous or missing. 
        if (quality not in acceptable_quality or height == missing):
            height = np.nan
        ceiling_height.append(height)
    data['Cloud_Height'] = ceiling_height
    return data

In [ ]:
def clean_visibility_data(data):
    '''The horizontal distance at which an object can be seen and identified. (meters)
    '''
    visilibilites = []
    acceptable_quality = (0,1,4,5,9) #see docs
    missing = 99999
    for i,vis in enumerate(data['VIS']):
        visibility = int(vis[0:6])
        quality = int(vis[7])
        #ID erroneuous or missing. 
        if (quality not in acceptable_quality or visibility == missing):
            visibility = np.nan
        visilibilites.append(visibility)
    data['Visibility'] = visilibilites
    return data

In [ ]:
def clean_temperature_data(data): 
    '''Air Temperature data in C'''
    temperatures = []
    missing = 9999
    acceptable_quality = ('0','1','4','5','9','C','I','M','P','R','U') #see docs
    for i,sample in enumerate(data['TMP']):
        temperature = int(sample[0:4])
        quality = sample[6]
        #ID erroneuous or missing. 
        if (quality not in acceptable_quality or temperature == missing):
            temperature = np.nan
        temperatures.append(temperature)
    data['Temperature'] = temperatures
    return data


In [ ]:
def clean_pressure_data(data): 
    '''The air pressure relative to Mean Sea Level (MSL).
    (Hectopascals)'''
    pressures = []
    missing = 99999
    acceptable_quality = ('0','1','4','5','9') #see docs
    for i,sample in enumerate(data['SLP']):
        pressure = int(sample[0:4])
        quality = sample[6]
        #ID erroneuous or missing. 
        if (quality not in acceptable_quality or pressure == missing):
            pressure = np.nan
        pressures.append(pressure)
    data['Atmospheric_Pressure'] = pressures
    return data


In [ ]:
def clean_dew_point_data(data): 
    '''The temperature to which a given parcel of air must be cooled
     at constant pressure and water vapor content in order for saturation to occur. (C)'''
    dew_points = []
    missing = 9999
    acceptable_quality = ('0','1','4','5','9','C','I','M','P','R','U') #see docs
    for i,sample in enumerate(data['DEW']):
        dp = int(sample[0:4])
        quality = sample[6]
        #ID erroneuous or missing. 
        if (quality not in acceptable_quality or dp == missing):
            dp = np.nan
        dew_points.append(dp)
    data['Dew_Point'] = dew_points
    return data

In [ ]:
def clean_precipitation_data(data): 
    '''episode of LIQUID-PRECIPITATION.
    - The quantity of time over which the LIQUID-PRECIPITATION was measured. (hours)
    - The depth of LIQUID-PRECIPITATION that is measured at the time of an observation. (mm)
    Note that there data contains AA1-AA3 fields for multiple precipitation events
    98% of samples do not have more than 1 event (AA2-AA4 are 98% nan), so these are ignored 
    TO-DO: Check above statement on total dataste
    To-DO: how to deal with nan data (no rain event)
    '''
    times, depths, to_drop = [], [], []
    missing_depth, missing_time = 9999, 99
    acceptable_quality = ('0','1','4','5','9','C','I','M','P','R','U') #see docs
    for i,sample in enumerate(data['AA1']):
        if isinstance(sample, str):
            time = int(sample[0:2])
            depth = int(sample[3:7])
            quality = sample[-1]
        else: 
            time = depth = quality = np.nan
        #ID erroneuous or missing. 
        if (quality not in acceptable_quality):
            depth = time = np.nan
        if (depth == missing_depth):
            depth = np.nan
        if (time == missing_time):
            time = np.nan
        depths.append(depth),  times.append(time)
    data['Precipitation_Duration'] = times
    data['Precipitation_Depth'] = depths
    return data


In [ ]:
def clean_sky_cover_data(data): 
    '''SKY-COVER-LAYER..
    - Field 1: The code that denotes the fraction of the total celestial dome covered by a SKY-COVER-LAYER.
    - Field 2: SKY-COVER-LAYER base height dimension
    - Field 3: The code that denotes the classification of the clouds that comprise a SKY-COVER-LAYER.
    Note that there data contains GA1-GA3 fields for multiple cloud layers
    GA2-GA3 are 89+% nan (no secondary cloud covered), so these are ignored 
    TO-DO: Check above statement on total dataset
    T0-DO: compare cloud cover fields and choose appropriate one(s) (GA1, GD1, GF1, )
    '''
    covers, height, cloud_type, to_drop = [], [], [], []
    acceptable_quality = ('0','1','4','5','9','M') #see docs
    missing_cover = 99
    #cloud cover measure in octas or code to fraction conversion



In [ ]:

i = 15
colname = df.columns[i]
print(colname)
print(df[colname].isna().sum())
print(df[colname].isna().sum() / len(df))

df.iloc[:, i].unique()

#fields
# Date, lat, long, wind_x, wind_y, cloud_height, Visibility, Temperature
# dew_point, Atmospheric_Pressure


SLP
0
0.0


array(['10151,5', '10157,5', '10161,5', '10167,5', '10173,5', '10174,5',
       '10177,5', '10170,5', '99999,9', '10175,5', '10163,5', '10171,5',
       '10172,5', '10182,5', '10179,5', '10169,5', '10137,5', '10134,5',
       '10128,5', '10126,5', '10123,5', '10116,5', '10118,5', '10122,5',
       '10120,5', '10112,5', '10115,5', '10117,5', '10131,5', '10140,5',
       '10144,5', '10147,5', '10138,5', '10129,5', '10153,5', '10160,5',
       '10166,5', '10181,5', '10184,5', '10188,5', '10191,5', '10195,5',
       '10196,5', '10199,5', '10204,5', '10214,5', '10225,5', '10233,5',
       '10236,5', '10226,5', '10221,5', '10220,5', '10228,5', '10240,5',
       '10245,5', '10250,5', '10253,5', '10256,5', '10255,5', '10249,5',
       '10251,5', '10248,5', '10257,5', '10264,5', '10268,5', '10274,5',
       '10275,5', '10267,5', '10238,5', '10237,5', '10241,5', '10246,5',
       '10247,5', '10244,5', '10242,5', '10239,5', '10254,5', '10258,5',
       '10234,5', '10222,5', '10217,5', '10213,5', 

In [ ]:
# fields to check with all the data
check = []
for i in range(len(df.columns)):
    colname = df.columns[i]
    if (df[colname].isna().sum() / len(df) < 0.9) and (df[colname].isna().sum() > 0.1):
        check.append(i)

In [ ]:
check

[16, 41, 43, 49, 50, 52, 53, 55, 56, 57, 58, 67, 68]

In [ ]:
def average_wind_vector(wind_data):
    '''
    Args: 
        wind_data: dataframe with wind direction and speed cols
    Returns: 
        tuple of mean x,y wind vectors
    '''
    #data is calibrated to north as 0 deg ?
    # wind_direction += 90
    wind_vect_x, wind_vect_y = [], []
    for dir,speed in wind_data.itertuples(index=False):
        #degrees to vectors
        wind_vect_x.append(np.cos(dir) * speed) #positive = west
        wind_vect_y.append(np.sin(dir) * speed) #positive = north
    wind_vect_x, wind_vect_y = np.array(wind_vect_x), np.array(wind_vect_y)
    wind_vect_x = wind_vect_x[~np.isnan(wind_vect_x)]
    wind_vect_y = wind_vect_y[~np.isnan(wind_vect_y)]
    return (np.mean(wind_vect_x), np.mean(wind_vect_y))
    
def average_cloud_height(df):
    cloud_height = np.array(df['Cloud_Height'])
    cloud_height = cloud_height[~np.isnan(cloud_height)]
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    sigmoid = np.vectorize(sigmoid)
    cloud_height = sigmoid(cloud_height)
    height = np.mean(cloud_height)
    return height

def mean_remove_nan(series):
    array = np.array(series)
    array = array[~np.isnan(array)]
    mean = np.mean(array)
    return mean


def average_daily_station_data(df_dict):
    '''Takes daily weather data dict and returns an average for that weather station
    TO-DO: 
        Average for other fields
    Args: 
        df_dict: dict of daily weather dataframes
    Returns: 
        Pandas dataframe of daily averages 
    '''
    avg_dict = {
        'Date': [], 
        'Latitude': [],
        'Longitude': [],
        'Wind_X': [],
        'Wind_Y': [],
        'Elevation': [],
        'Cloud_Height':[],
        'Temperature':[],
        'Visibility':[],
        'Atmospheric_Pressure':[],
        'Dew_Point':[],
        'Precipitation_Duration':[],
        'Precipitation_Depth':[],
        # 'Cloud_Cover':[]
    }
    for date, df in df_dict.items():
        assert len(df_dict[date]['LATITUDE'].unique()) == 1
        assert len(df_dict[date]['LONGITUDE'].unique()) == 1
        assert len(df_dict[date]['ELEVATION'].unique()) == 1
        avg_dict['Date'].append(date)
        avg_dict['Latitude'].append(df['LATITUDE'].iloc[0])
        avg_dict['Longitude'].append(df['LONGITUDE'].iloc[0])
        avg_dict['Elevation'].append(df['ELEVATION'].iloc[0])

        df = clean_wind_data(df)
        wind_x, wind_y = average_wind_vector(df[['Wind_Direction', 'Wind_Speed']])
        avg_dict['Wind_X'].append(wind_x)
        avg_dict['Wind_Y'].append(wind_y)

        df = clean_ceiling_height_data(df)
        height = average_cloud_height(df)
        avg_dict['Cloud_Height'].append(height)

        df = clean_visibility_data(df)
        vis = mean_remove_nan(df['Visibility'])
        avg_dict['Visibility'].append(vis)

        df = clean_temperature_data(df)
        temp = mean_remove_nan(df['Temperature'])
        avg_dict['Temperature'].append(temp)
          
        df = clean_pressure_data(df)
        pressure = mean_remove_nan(df['Atmospheric_Pressure'])
        avg_dict['Atmospheric_Pressure'].append(pressure)

        df = clean_dew_point_data(df)
        dp = mean_remove_nan(df['Dew_Point'])
        avg_dict['Dew_Point'].append(dp)
          
        df = clean_precipitation_data(df)
        dur = mean_remove_nan(df['Precipitation_Duration'])
        avg_dict['Precipitation_Duration'].append(dur)
        depth = mean_remove_nan(df['Precipitation_Depth'])
        avg_dict['Precipitation_Depth'].append(depth)

        #TO-DO: add cloud cover

    return pd.DataFrame(avg_dict)


In [ ]:
average_daily_station_data(df_dict)

,Date,Latitude,Longitude,Wind_X,Wind_Y,Elevation,Cloud_Height,Temperature,Visibility,Atmospheric_Pressure,Dew_Point,Precipitation_Duration,Precipitation_Depth
0,2020-01-01,34.20056,-118.3575,5.586300,8.545516,225.9,1.0,85.592593,88915.333333,1681.259259,73.296296,1.884615,0.000000
1,2020-01-02,34.20056,-118.3575,-2.736079,-4.847060,225.9,1.0,52.600000,55449.240000,1371.560000,44.960000,1.920000,0.000000
2,2020-01-03,34.20056,-118.3575,-2.126164,-4.126998,225.9,1.0,52.680000,55449.240000,1378.640000,43.480000,1.920000,0.000000
3,2020-01-04,34.20056,-118.3575,0.105888,18.686145,225.9,1.0,52.080000,55449.240000,1383.600000,42.920000,1.920000,0.000000
4,2020-01-05,34.20056,-118.3575,-5.908712,-2.317946,225.9,1.0,50.538462,53935.538462,1714.038462,40.230769,1.920000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2020-12-27,34.20056,-118.3575,-7.135497,-2.780202,225.9,1.0,36.794872,34450.846154,4470.076923,32.794872,1.920000,0.000000
362,2020-12-28,34.20056,-118.3575,6.228651,5.308263,225.9,1.0,26.305085,29809.932203,6342.491525,23.762712,1.511111,10.311111
363,2020-12-29,34.20056,-118.3575,3.852979,14.682670,225.9,1.0,84.346154,53687.961538,2397.500000,79.076923,1.884615,10.038462
364,2020-12-30,34.20056,-118.3575,-0.165417,2.797364,225.9,1.0,48.769231,53935.538462,1711.807692,34.846154,1.920000,0.000000
